<a href="https://colab.research.google.com/github/abisubramanya27/ChAII-docker/blob/master/ChAII_COPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Env Setup (Check out https://towardsdatascience.com/conda-google-colab-75f7c867a522 for detailed instructions)
BEFORE STARTING, SET RUNTIME_TYPE TO GPU. 

# ChAII starter notebook

This is a starter notebook for running a baseline mBERT model on the task. This is a standalone notebook which will allow you to do the following:
1. Train an mBERT model on the ChAII data (utilizing Colab GPUs), 
2. Get dev evaluation numbers, 
3. Generate a submission for the Kaggle leaderboard with the appropriate format.

This notebook uses the [Xtreme](https://github.com/google-research/xtreme) codebase for training QA-finetuned models. Feel free to run your own scripts/variants locally, experiment with other models and pipelines, not necessarily limited to Xtreme. 

In [1]:
# Verify PYTHONPATH is blank to avoid problems later
!echo $PYTHONPATH # should return <blank>

In [2]:
# Verify that Miniconda installation and updation worked

!which conda # should return /usr/local/bin/conda
!conda --version # should return 4.10.3
!which python # should return /usr/local/bin/python
!python --version # should return Python 3.6.13 :: Anaconda, Inc.

/opt/conda/bin/conda
conda 4.10.3
/opt/conda/bin/python
Python 3.7.11


In [3]:
# Overview of path files

import sys
sys.path

['/root/mount',
 '/opt/conda/lib/python37.zip',
 '/opt/conda/lib/python3.7',
 '/opt/conda/lib/python3.7/lib-dynload',
 '',
 '/opt/conda/lib/python3.7/site-packages',
 '/opt/conda/lib/python3.7/site-packages/IPython/extensions',
 '/root/.ipython']

## Training

Although this codebase can be used for many varieties of training methods and experiments, we will only train a straightforward baseline. We will create a monolingual Hindi QA model. We encourage you to read and experiment with the Xtreme codebase, and also with other repos. Some promising avenues:

* Train model on both Hindi and Tamil ChAII data,
* Multi-task learning with Xtreme,
* Annotate your own data into a QA format and augment training,
* Zero-shot transfer learning

The cells below do the following:

1. Convert the given ChAII data (from competition) to QA (SQuAD) format, split into train and dev sets.
2. Finetune mBERT (bert-base-multilingual-cased) on the ChAII data.
3. Save the model and dev predictions into GDrive for evaluation later

In [1]:
# Load ChAII dataset

import os
import collections
import functools
import glob
import json
import random
from pprint import pprint
from tqdm import tqdm
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", 20, "display.max_columns", None)

data_path = Path("/root/mount/dataset")
json_dicts = []

def get_dataframe(file_path):
    df = pd.DataFrame()
    with open(file_path,'r') as f:
        df = pd.read_csv(f)
    df = df.astype(str)
    df = df.apply(lambda x: x.str.strip())
    return df

chaii_data = get_dataframe(data_path / "train.csv")
chaii_data.head

<bound method NDFrame.head of              id                                            context  \
0     903deec17  ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...   
1     d9841668c  காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...   
2     29d154b56  சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...   
3     41660850a  குழந்தையின் அழுகையை  நிறுத்தவும், தூங்க வைக்கவ...   
4     b29c82c22  சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...   
5     d419db018  மின்னணுவியல் (Electronics) மின்னணுக்கள் அல்லது...   
6     c29e29ab6  அலெக்ஸாண்டர் கிரகாம் பெல் ( Alexander Graham B...   
7     2a5ba78e2  முதல் உலகப்போர் என்பது உலகம் தழுவிய அளவில் இடம...   
8     10ff95f4c  இந்தியாவின் தேசிய மனித உரிமை ஆணையம்  ஒரு தன்னா...   
9     7a6e807d7  நெல்சன் மண்டேலா (Nelson Rolihlahla Mandela, 18...   
...         ...                                                ...   
1104  187813eeb  फ़ख़रुद्दीन अली अहमद (3 मई 1905 - 11 फरवरी 197...   
1105  e0403a678  सत्येन्द्र नारायण सिन्हा (12 जुलाई 1917 – 4

### Data conversion to QA format

The below cells convert TyDiQA and ChAII Kaggle data format to the SQuAD QA format, so it can be used with the Xtreme pipeline. You need to download the ChAII data from Kaggle, and put it in the mount/dataset directory.

In [2]:
# TODO: Convert from Kaggle training format to QA format
language = 'hindi' # replace this with tamil when processing tamil dataset
lang_code = 'hi' # for tamil use ta

def convert_to_qa_format_kaggle(row):
    answer = {}
    answer["text"] = row["answer_text"]
    answer["answer_start"] = int(row["answer_start"])
    qa_json = {
        "title": "",
        "paragraphs": [
            {
                "context": row["context"],
                "qas": [
                    {
                        "question": row["question"],
                        "id": row["language"] + '-' + str(row["id"]),
                        "answers": [answer]
                    }
                ]
            }
        ],
    }
    
    return qa_json

# Process one language at a time
# Here chaii_data is a pandas dataframe
def get_qa_data_from_kaggle_format(chaii_data, language): 
    language = 'hindi'

    qa_data = {"data":[], "version":f"chaii_{language}"}
    for index, row in chaii_data.iterrows():
        if row["language"] == language:
            qa_datapoint = convert_to_qa_format_kaggle(row)
            qa_data["data"].append(qa_datapoint)

    print("QA (SQuAD) format:")
    print(qa_data["data"][0])
    return qa_data

qa_data = get_qa_data_from_kaggle_format(chaii_data, language)

QA (SQuAD) format:
{'title': '', 'paragraphs': [{'context': 'विषाणु अकोशिकीय अतिसूक्ष्म जीव हैं जो केवल जीवित कोशिका में ही वंश वृद्धि कर सकते हैं।[1] ये नाभिकीय अम्ल और प्रोटीन से मिलकर गठित होते हैं, शरीर के बाहर तो ये मृत-समान होते हैं परंतु शरीर के अंदर जीवित हो जाते हैं। इन्हे क्रिस्टल के रूप में इकट्ठा किया जा सकता है। एक विषाणु बिना किसी सजीव माध्यम के पुनरुत्पादन नहीं कर सकता है। यह सैकड़ों वर्षों तक सुशुप्तावस्था में रह सकता है और जब भी एक जीवित मध्यम या धारक के संपर्क में आता है उस जीव की कोशिका को भेद कर आच्छादित कर देता है और जीव बीमार हो जाता है। एक बार जब विषाणु जीवित कोशिका में प्रवेश कर जाता है, वह कोशिका के मूल आरएनए एवं डीएनए की जेनेटिक संरचना को अपनी जेनेटिक सूचना से बदल देता है और संक्रमित कोशिका अपने जैसे संक्रमित कोशिकाओं का पुनरुत्पादन शुरू कर देती है।\nविषाणु का अंग्रेजी शब्द वाइरस का शाब्दिक अर्थ विष होता है। सर्वप्रथम सन १७९६ में डाक्टर एडवर्ड जेनर ने पता लगाया कि चेचक, विषाणु के कारण होता है। उन्होंने चेचक के टीके का आविष्कार भी किया। इसके बाद सन १८८६ में एडो

In [3]:
print("Total number of datapoints: %d" % len(qa_data["data"]))

Total number of datapoints: 746


In [4]:
# Split datapoints language-wise and into QA format
# Run this cell only if you need to convert from TyDiQA to SQuAD format, otherwise run the nexy one.
import re

from pprint import pprint

def byte_str(text):
  return text.encode("utf-8")

def byte_len(text):
  # Python 3 encodes text as character sequences, not byte sequences
  # (like Python 2).
  return len(byte_str(text))

def byte_slice(text, start, end, errors="replace"):
  # Python 3 encodes text as character sequences, not byte sequences
  # (like Python 2).
  return byte_str(text)[start:end].decode("utf-8", errors=errors)

def convert_to_qa_format_tydiqa(tydi_json):
  answer = {}
  for annotation in tydi_json["annotations"]:
    minimal_answer = annotation["minimal_answer"]
    if minimal_answer["plaintext_start_byte"] != -1 and minimal_answer["plaintext_end_byte"] != -1:
      answer["text"] = byte_slice(tydi_json["document_plaintext"],minimal_answer["plaintext_start_byte"],minimal_answer["plaintext_end_byte"])
      answer["answer_start"] = [m.start() for m in re.finditer(answer["text"],tydi_json["document_plaintext"])][0]
      break
  if answer == {}:
    return {}
  
  qa_json = {
      "title" : tydi_json["document_title"],
      "paragraphs" : [
                      {
                          "context": tydi_json["document_plaintext"],
                          "qas" : [
                                   {
                                    "question" : tydi_json["question_text"],
                                    "id" : tydi_json["language"] + '-' + str(tydi_json["example_id"]),
                                    "answers" : [answer],
                                   }
                          ]
                      }
      ],
  }

  return qa_json

# Here chaii_data is json list
def get_qa_data_from_tydiqa_format(chaii_data, language):
    language = 'hindi'
    qa_data = {"data":[], "version":f"chaii_{language}"}
    for json_dict in json_dicts:
      if json_dict["language"] == language:
        qa_datapoint = convert_to_qa_format_tydiqa(json_dict)
        if qa_datapoint != {}:
          qa_data["data"].append(qa_datapoint)
        qa_data['data'].append(json_dict)

    print("QA (SQuAD) format:")
    print(qa_data["data"][0])

In [5]:
# Splitting data into train and dev and saving converted QA formats

import random

qa_data_datapoints = qa_data["data"]
random.shuffle(qa_data_datapoints)
train_size = int(len(qa_data_datapoints)*0.8)
train_qa_data_datapoints, dev_qa_data_datapoints = qa_data_datapoints[:train_size], qa_data_datapoints[train_size:]

train_qa_data = {"data":train_qa_data_datapoints, "version":f"chaii_{lang_code}_train"}
dev_qa_data = {"data":dev_qa_data_datapoints, "version":f"chaii_{lang_code}_dev"}

with open(os.path.join(data_path,f"train.{lang_code}.qa.jsonl"),'w') as f:
  json.dump(train_qa_data,f)

with open(os.path.join(data_path,f"dev.{lang_code}.qa.jsonl"),'w') as f:
  json.dump(dev_qa_data,f)

print("Training data size: %d" % len(train_qa_data_datapoints))
print("Dev data size: %d" % len(dev_qa_data_datapoints))

Training data size: 596
Dev data size: 150


The cell below is optional (we have not used it for our baseline model), but it downloads the original TyDiQA data in the QA format. You can combine it with our ChAII data and boost training!

In [6]:
%%bash
# Downloading the data. For baseline, we can ignore the other training datasets for other tasks, and focus on training with just TyDiQA data. 

source activate xtreme
cd /root/xtreme # Optional but recommended

# Copyright 2020 Google and DeepMind.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

REPO=$PWD
DIR=$REPO/download/
mkdir -p $DIR

function download_tydiqa {
    echo "download tydiqa-goldp"
    base_dir=$DIR/tydiqa/
    mkdir -p $base_dir && cd $base_dir
    tydiqa_train_file=tydiqa-goldp-v1.1-train.json
    tydiqa_dev_file=tydiqa-goldp-v1.1-dev.tgz
    wget https://storage.googleapis.com/tydiqa/v1.1/${tydiqa_train_file} -q --show-progress
    wget https://storage.googleapis.com/tydiqa/v1.1/${tydiqa_dev_file} -q --show-progress
    tar -xf ${tydiqa_dev_file}
    rm ${tydiqa_dev_file}
    out_dir=$base_dir/tydiqa-goldp-v1.1-train
    python $REPO/utils_preprocess.py --data_dir $base_dir --output_dir $out_dir --task tydiqa
    mv $base_dir/$tydiqa_train_file $out_dir/
    echo "Successfully downloaded data at $DIR/tydiqa" >> $DIR/download.log
}

download_tydiqa

Process is interrupted.


### Training mBERT on Hindi ChAII data

The below script uses the Xtreme script to train the data. Here, we need to modify the code in the folders to train it on the ChAII data. You can double click on the scripts, modify the code and change them. 

For the baseline, the following changes were made to the Xtreme repo code:


1.   In ```scripts/train.sh```, an additional task called "chaii_hi" was added as such:
```
...
elif [ $TASK == 'chaii_hi' ]; then
  bash $REPO/scripts/train_qa.sh $MODEL chaii_hi $TASK $GPU $DATA_DIR $OUT_DIR
...
```
2.   In ```scripts/train_qa.sh```, the following flags were added:
```
TRAIN_LANG="en"
EVAL_LANG="en"
```
Another elif condition was added as such to modify path of data dir:
```
...
elif [ $SRC == 'chaii_hi' ]; then
  TASK_DATA_DIR=${DATA_DIR}
  TRAIN_FILE=${TASK_DATA_DIR}/train.hi.qa.jsonl
  PREDICT_FILE=${TASK_DATA_DIR}/dev.hi.qa.jsonl
  TRAIN_LANG="hi"
  EVAL_LANG="hi"
...
```
Finally, TRAIN_LANG and EVAL_LANG replaced the hardcoded "en":
```
 --weight_decay 0.0001 \
  --threads 8 \
  --train_lang ${TRAIN_LANG} \
  --eval_lang ${EVAL_LANG}
```

If you want to make your own changes for experimentation, clone the xtreme repo locally in the mount folder and mount it as part of the docker container. 

Finally, we create a run.sh script in the current root directory, and paste the following commands:

```
#!/bin/bash

TASK=${1:-chaii_hi}
DATA_DIR=${2:-"/root/xtreme/download/chaii_data/"}
OUT_DIR=${3:-"/root/xtreme/outputs-temp/"}
MODEL=${4:-bert-base-multilingual-cased}
GPU=${5:-0}
TRAIN_FILE_NAME=${6}
PREDICT_FILE_NAME=${7}

source activate xtreme
cd /root/xtreme
bash scripts/train.sh $MODEL $TASK $GPU $DATA_DIR $OUT_DIR $TRAIN_FILE_NAME $PREDICT_FILE_NAME

```

Your model should be stored in ```/root/mount/outputs-temp/```.

In [8]:
!pip install ipython-autotime
%load_ext autotime

# We store all the outputs and models to our local mount directory 
# For tamil change the task to 'chaii_ta'
!bash /root/run.sh chaii_hi "/root/mount/dataset" "/root/mount/outputs-temp" bert-base-multilingual-cased 0 train.hi.qa.jsonl dev.hi.qa.jsonl

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
Fine-tuning bert-base-multilingual-cased on chaii_hi using GPU 0
Load data from /root/mount/dataset, and save models to /root/mount/outputs-temp
08/09/2021 15:43:28 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
08/09/2021 15:43:28 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at /root/.cache/torch/transformers/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0
08/09/2021 15:43:28 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "do_sample": false,
  "eos_token_ids": 

Iteration:   6%|█▋                           | 291/4898 [00:54<13:54,  5.52it/s]


Iteration:  10%|██▊                          | 485/4898 [01:30<13:48,  5.32it/s]


Iteration:  14%|████                         | 679/4898 [02:06<12:44,  5.52it/s]


Iteration:  18%|█████▏                       | 873/4898 [02:42<12:36,  5.32it/s]


...

Iteration:  88%|████████████████████████▋   | 4316/4898 [13:23<01:51,  5.23it/s]


Iteration:  92%|█████████████████████████▊  | 4510/4898 [13:59<01:11,  5.45it/s]


Iteration:  96%|██████████████████████████▉ | 4704/4898 [14:35<00:37,  5.23it/s]


Epoch: 100%|█████████████████████████████████████| 2/2 [30:23<00:00, 911.63s/it]
08/09/2021 16:15:44 - INFO - __main__ -    global_step = 2449, average loss = 0.39099916937107293
08/09/2021 16:15:44 - INFO - __main__ -   Saving model checkpoint to /root/mount/outputs-temp/chaii_hi/bert-base-multilingual-cased_LR3e-5_EPOCH2.0_maxlen384


08/09/2021 16:15:44 - INFO - transformers.configuration_utils -   Configuration saved in /root/mount/outputs-temp/chaii_hi/bert-base-multilingual-cased_LR3e-5_EPOCH2.0_maxlen384/config.json
08/09/2021 16:15:48 - INFO - transformers.modeling_utils -   Model weights saved in /root/mount/outputs-temp/chaii_hi/bert-base-multilingual-cased_LR3e-5_EPOCH2.0_maxlen384/pytorch_model.bin
08/09/2021 16:15:48 - INFO - transformers.configuration_utils -   loading configuration file /root/mount/outputs-temp/chaii_hi/bert-base-multilingual-cased_LR3e-5_EPOCH2.0_maxlen384/config.json
08/09/2021 16:15:48 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABE

## Inference and Evaluation

For inference, we do the following modifications to Xtreme repo:
1. In ```predict_qa.sh```, add the following (line 40):
```
elif [ $TGT == 'chaii_hi' ]; then
  langs=( hi )
```


Also, we create a bash file (similar to ```run.sh```) called ```predict.sh```, and copy the commands below into it:

```
#!/bin/bash

source activate xtreme
cd /root/xtreme

MODEL_PATH=${1:-"/root/xtreme/outputs-temp/chaii_hi/bert-base-multilingual-cased_LR3e-5_EPOCH2.0_maxlen384"}
TASK=${2:-chaii_hi}
DATA_DIR=${2:-"/root/xtreme/download/chaii_data/"}
PREDICTIONS_DIR=${3:-"/root/xtreme/predictions/"}
MODEL=${4:-bert-base-multilingual-cased}
MODEL_TYPE=${5:-bert}
GPU=${6:-0}
PREDICT_FILE_NAME=${7}
 
bash scripts/predict_qa.sh bert-base-multilingual-cased bert $MODEL_PATH $TASK $GPU $DATA_DIR $PREDICTIONS_DIR $PREDICT_FILE_NAME
```

In [9]:
# First, you need to run inference on the models

!bash /root/predict.sh "/root/mount/outputs-temp/chaii_hi/bert-base-multilingual-cased_LR3e-5_EPOCH2.0_maxlen384" \
      chaii_hi "/root/mount/dataset" "/root/eval_dir/predictions" 

************************
bert-base-multilingual-cased
************************

Predictions on chaii_hi
  hi 
time: 11.9 s (started: 2021-08-09 16:30:30 +00:00)


In [10]:
def jaccard(str1, str2): 
    a = set(str1.lower().split()) 
    b = set(str2.lower().split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

def evaluate(lang_code):
    # For evaluating the predictions, we will use our custom script which uses jaccard mean 
    import json
    with open(f"/root/mount/outputs-temp/chaii_{lang_code}/bert-base-multilingual-cased_LR3e-5_EPOCH2.0_maxlen384/predictions_{lang_code}_.json") as f:
      preds = json.load(f)

    with open(f"/root/mount/dataset/dev.{lang_code}.qa.jsonl") as f:
      dev_data = json.load(f)
    
    submission_preds = [{'id':k.split('-')[1], 'PredictionString': v} for k, v in preds.items()]
    
    # write submissions file
    df_ = pd.DataFrame.from_dict(submission_preds)
    df_.to_csv(f'/root/mount/eval_dir/chaii_{lang_code}_submission.csv', index=False)
    
    from pprint import pprint
    jaccard_mean = 0
    dev_answer_pair_matches = []
    for d in dev_data['data']:
        for para in d['paragraphs']:
            for qa in para['qas']:
                sample_jaccard = jaccard(qa['answers'][0]['text'], preds[qa['id']])
                jaccard_mean += sample_jaccard
                dev_answer_pair_matches.append({'context':para['context'],'question':qa['question'],'gold_answer':qa['answers'],'mbert_pred':preds[qa['id']],'id':qa['id']})

    jaccard_mean /= len(dev_answer_pair_matches)
    print(f"Jaccard Mean for chaii_{lang_code}: {jaccard_mean}")
    
    return dev_answer_pair_matches

!mkdir /root/mount/eval_dir
dev_answer_pair_matches_hi = evaluate("hi")

mkdir: cannot create directory ‘/root/mount/eval_dir’: File exists
Jaccard Mean for chaii_hi: 0.4174314574314575
time: 752 ms (started: 2021-08-09 16:30:47 +00:00)


In [11]:
def write_dev_answer_pair_matches(dev_answer_pair_matches, lang_code):
    #Matches in predictions
    correct_ans = [d for d in dev_answer_pair_matches if d['mbert_pred']==d['gold_answer'][0]['text']]
    with open(f'/root/mount/eval_dir/correct_chaii_{lang_code}_mbert.txt','w',encoding='utf-8') as f:
      for c in correct_ans:
        f.write(f"id:{c['id']}\n")
        f.write(f"context:{c['context']}\n")
        f.write(f"question:{c['question']}\n")
        f.write(f"gold_answer:{c['gold_answer'][0]['text']}\n")
        f.write(f"mbert_pred:{c['mbert_pred']}\n")
        f.write("\n\n")
        
    #Mismatches in predictions
    wrong_ans = [d for d in dev_answer_pair_matches if d['mbert_pred']!=d['gold_answer'][0]['text']]
    with open(f'/root/mount/eval_dir/wrong_chaii_{lang_code}_mbert.txt','w',encoding='utf-8') as f:
      for c in wrong_ans:
        f.write(f"id:{c['id']}\n")
        f.write(f"context:{c['context']}\n")
        f.write(f"question:{c['question']}\n")
        f.write(f"gold_answer:{c['gold_answer'][0]['text']}\n")
        f.write(f"mbert_pred:{c['mbert_pred']}\n")
        f.write("\n\n")
    
    return correct_ans, wrong_ans
        
correct_ans, wrong_ans = write_dev_answer_pair_matches(dev_answer_pair_matches_hi, "hi")

time: 24.1 ms (started: 2021-08-09 16:30:53 +00:00)


In [12]:
len(correct_ans),len(wrong_ans)

(51, 99)

time: 2.32 ms (started: 2021-08-09 16:30:53 +00:00)
